In [1]:
# coding: utf-8
# this will download audioset youtube samples based on the audioset segments
# 
# balanced
# unbalanced
# eval

import numpy as np
import pandas as pd
import re
import csv
import datetime
import os
import wave

seg_path = '../labels/audioset/segments/'
#segment = 'balanced_train'
segment = 'unbalanced_train'
#segment = 'eval'

## Lees csv-file in
labels = pd.read_csv(seg_path+segment+"_segments.csv", skiprows=3, 
                          quotechar='"', skipinitialspace = True, header=None, 
                          names = ["YTID", "start_seconds", "end_seconds", "positive_labels"])


In [2]:
labels.head()

,YTID,start_seconds,end_seconds,positive_labels
0,---1_cCGK4M,0.0,10.0,"/m/01g50p,/m/0284vy3,/m/06d_3,/m/07jdr,/m/07rwm0c"
1,---2_BBVHAA,30.0,40.0,/m/09x0r
2,---B_v8ZoBY,30.0,40.0,/m/04rlf
3,---EDNidJUA,30.0,40.0,"/m/02qldy,/m/02zsn,/m/05zppz,/m/09x0r"
4,---N4cFAE1A,21.0,31.0,"/m/04rlf,/m/09x0r"


In [3]:
## op pcloud is er een symlink naar de data folder
# downloaded segments go to
target_file_path = "../../../data/audio/serval-data/raw/youtube-downloads/"+segment+"/"
#target_file_path = "./"+segment+"/"

# welke custom serval wordt er getraned
custom_target = "jungle" #"urban"

# we need a list of custom class labels
mid_to_label = pd.read_csv("../labels/custom/"+custom_target+"/class_labels_indices_jungle_v2.csv", sep=";")

mid_to_label

,index,mid,display_name
0,0,/t/dd00134,"Car, boat, motorcyle, truck"
1,1,/m/01j4z9,Chainsaw
2,2,/m/07pczhz,Chop
3,3,/m/032s66,"Gunshot, gunfire"
4,4,/m/09ct_,Helicopter
5,5,/t/dd00003,Male singing
6,6,/m/07rpkh9,Cattle
7,0,/m/02rlv9,"Car, boat, motorcyle, truck"
8,0,/m/04_sv,"Car, boat, motorcyle, truck"
9,7,/m/06bz3,Radio


In [4]:
# define functions
# find unique set of labels in custom target

def getLabels(mid_str):
    ## Maak lijst van m-id's
    mid_list = mid_str.split(',')
    #print(mid_list)
    labels = []
    
    ## Voor elk m-id, vind labels, (if any labels: add to label list)
    for mid in mid_list:
        if (mid_to_label.loc[mid_to_label["mid"] == mid, "mid"].any()):
            labels.append(mid_to_label.loc[mid_to_label["mid"] == mid, "index"].values[0])
    
    ## Return unique set of labels
    return set(labels)


In [5]:
# Download wav-files van youtube

def downloadWav(vid_to_mid, path):
    # now we actually download the segments as they are no conversion yet
    mids = vid_to_mid["positive_labels"]
    labels = getLabels(mids)
    #print('aantal labeld : ',len(labels))
    
    if(len(labels)>0):
        url = vid_to_mid["YTID"]
        start_sec = vid_to_mid["start_seconds"]
        start_time = str(datetime.timedelta(seconds=start_sec)) + '.00'
        file_path = path + "vid" + url 
        # TODO download only if not exists
        if not os.path.exists(file_path):
            # we only download the 10 second segment as is
            cmd = 'youtube-dl -f "bestaudio" -o "' + file_path + '.%(ext)s" --extract-audio  --postprocessor-args "-ss ' + start_time + ' -t 00:00:10.00" "https://www.youtube.com/v/' + url + '"'
            #print("CMD="+cmd)
            os.system(cmd)
        else:
             print('File already exists!')

In [ ]:
# add progressbar
from tqdm import tqdm

from joblib import Parallel, delayed
import multiprocessing


num_cores = multiprocessing.cpu_count()

# run in parralel
results = Parallel(n_jobs=num_cores)(delayed(downloadWav)(labels.loc[i], target_file_path) for i in tqdm(range(labels.shape[0])))

# not paralel
#for i in tqdm(range(labels.shape[0])):
# for i in range(labels.shape[0]):
#     downloadWav(labels.loc[i], target_file_path)


  8%|▊         | 170178/2041789 [59:24<10:53:26, 47.74it/s]